# importing libraries and defining directories

Importing libraries


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,models
from torchvision import transforms as T
import torch.nn.functional as F
import cv2
import random

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


defining the directories


In [ ]:
test_dir="/content/drive/MyDrive/projet deep learning/test"
queries_dir="/content/drive/MyDrive/projet deep learning/queries"
train_dir="/content/drive/MyDrive/projet deep learning/train"

# Model functions

In [ ]:
class model_ft(torch.nn.Module):
  def __init__(self):
    super(model_ft, self).__init__()
    model_resnet = models.resnet50(pretrained=True)
    num_ftrs = model_resnet.fc.in_features
    model_resnet.fc = torch.nn.Sequential()
    self.model_resnet=model_resnet
    self.fc=torch.nn.Linear(num_ftrs,751)
  def forward(self, x1):
    x1 = self.model_resnet(x1)
    x1= self.fc(x1)
    return x1
  def embed(self, x1):
    x1 = self.model_resnet(x1)
    return x1 

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [ ]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
  return optimizer

# Training

organising train data in dictionary


In [ ]:
d_train=dict()
for filename in os.listdir(train_dir):
      id_img=int(filename.split("_")[0])
      if id_img in d_train.keys():
        d_train[id_img].append(filename)
      else:
        d_train[id_img]=[filename]

creating a training list

In [ ]:
keys_list=np.array(list(d_train.keys()))
full_train_list=[]
for i in range(len(keys_list)):
  for img in d_train[keys_list[i]]:
    ele=[img , i]
    full_train_list.append(ele)

function to create the dataset and get the data loader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, l, root_dir, transform=None):
        self.l =l
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.l)

    def __getitem__(self, idx):
        img_path=os.path.join(self.root_dir,self.l[idx][0])
        filename=self.l[idx][0]
        label=self.l[idx][1]
        img=cv2.imread(img_path)
        img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if self.transform:
            img = self.transform(img)
        return (img,label,filename)

In [ ]:
def get_data(batch_size,l,root_dir):
  transform = list()
  transform.append(T.ToTensor())
  transform.append(T.Resize((224,224)))
  transform.append(T.RandomHorizontalFlip(p=0.4)) 
  transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))           
  transform = T.Compose(transform)
  full_training_data = MyDataset(l,root_dir, transform=transform)
  # Create train and validation splits
  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.7+1)
  validation_samples = num_samples - training_samples
  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=1)
  validation_loader=torch.utils.data.DataLoader(validation_data, batch_size, shuffle=True, num_workers=1)
  return training_data, validation_data,train_loader,validation_loader

the training function

In [ ]:
def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (anchors,labels,filename) in enumerate(data_loader):
    # Load data into GPU
    anchors = anchors.to(device)
    labels=labels.to(device)
    # Forward pass
    anchors_out = net(anchors)
    # Apply the loss
    loss = cost_function(anchors_out,labels)
    # Backward pass
    loss.backward()
    # Update parameters
    optimizer.step()
    # Reset the optimizer
    optimizer.zero_grad()
    samples+=anchors.shape[0]
    cumulative_accuracy+=( (torch.argmax(anchors_out, dim=1)==labels)*1 ).sum()
    # Better print something, no?
    print(samples)
    cumulative_loss += loss.item()
  return cumulative_loss/samples,cumulative_accuracy/samples

Training

In [ ]:
batch_size=32
device='cuda:0' 
learning_rate=0.0003
weight_decay=0.000001
momentum=0.9
epochs=25
training_data, validation_data,train_loader,validation_loader = get_data(batch_size,full_train_list,train_dir)
net = model_ft().to(device)
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
cost_function = get_cost_function()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


In [ ]:
print('start training')
accuracy_per_epoch=[]
loss_per_epoch=[]
for e in range(epochs):
    train_loss,train_accuracy = train(net, train_loader, optimizer, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('train loss: ',train_loss)
    print('train_accuracy: ',train_accuracy*100)
    loss_per_epoch.append(train_loss)
    accuracy_per_epoch.append(train_accuracy)

start training
32.0
64.0


KeyboardInterrupt: ignored

In [ ]:
epochs=[i+1 for i in range(25)]
plt.plot(epochs,accuracy_per_epoch)
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(epochs,loss_per_epoch)
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets,filename) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()
      print(samples)
  return cumulative_loss/samples, cumulative_accuracy/samples*100

In [ ]:
val_loss,val_accuracy=test(net,validation_loader,cost_function)
print("loss for validation: ",val_loss)
print("accuracy for validation: ",val_accuracy)

Saving the model after training


In [ ]:
torch.save(net.state_dict(), "/content/drive/MyDrive/projet deep learning/person_reid_model_2")

# Testing Person re-id for the validation dataset

loading our model

In [ ]:
device='cuda:0' 
net = model_ft().to(device)
net.load_state_dict(torch.load("/content/drive/MyDrive/projet deep learning/person_reid_model_2"), strict=True)

<All keys matched successfully>

In [ ]:
class testing_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, l, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.l =l
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.l)

    def __getitem__(self, idx):
        img_path=os.path.join(self.root_dir,  self.l[idx])
        img=cv2.imread(img_path)
        img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
def get_test_data(batch_size,l,root_dir):
  transform = list()
  transform.append(T.ToTensor())
  transform.append(T.Resize((224,224)))                             # converts Numpy to Pytorch Tensor
  transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))                         
     # Normalizes the Tensors 
  transform = T.Compose(transform)
  data = testing_Dataset(l,root_dir, transform=transform)
  data_loader = torch.utils.data.DataLoader(data, batch_size, shuffle=False, num_workers=1)
  return data_loader

In [ ]:
def image_embed(l,root_dir,net,device):
  data_loader = get_test_data(1,l,root_dir)
  img_embd_arr=np.zeros( ( len(l) , 2048) )
  net.eval()
  with torch.no_grad():
    for batch_idx ,img in enumerate(data_loader):
      input=img.to(device)
      img_embd=net.embed(input)
      img_embd=img_embd.cpu().detach().numpy()
      img_embd_arr[batch_idx]=img_embd
      if (batch_idx % 256 ==0) :
        print(batch_idx)
  return torch.tensor(img_embd_arr).to(device)

In [ ]:
def predict(query_l,queries_dir,test_l,test_dir,net,device):
  img_embd_query=image_embed(query_l,queries_dir,net,device)
  print('query_embedding_finished')
  img_embd_test=image_embed(test_l,test_dir,net,device)
  print('test_embedding_finished')
  cos=torch.nn.CosineSimilarity(dim=1)
  d={ key:[] for key in query_l }
  for j in range( len(query_l) ):
      bo =np.array( cos(img_embd_query[j:j+1],img_embd_test) > 0.86 ) 

      d[ query_l[j] ].append(test_l[i])
  return d,img_embd_query,img_embd_test

In [ ]:
from typing import Dict, Set, List
def evaluate_map(predictions: Dict[str, List], ground_truth: Dict[str, Set]):
        '''
        Computes the mAP (https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173) of the predictions with respect to the given ground truth
        In person reidentification mAP refers to the mean of the AP over all queries.
        The AP for a query is the area under the precision-recall curve obtained from the list of predictions considering the
        ground truth elements as positives and the other ones as negatives

        :param predictions: dictionary from query filename to list of test image filenames associated with the query ordered
                            from the most to the least confident prediction.
                            Represents the predictions to be evaluated.
        :param ground_truth: dictionary from query filename to set of test image filenames associated with the query
                             Represents the ground truth on which to evaluate predictions.

        :return:
        '''

        m_ap = 0.0
        for current_ground_truth_query, current_ground_truth_query_set in ground_truth.items():

            # No predictions were performed for the current query, AP = 0
            if not current_ground_truth_query in predictions:
                continue

            current_ap = 0.0  # The area under the curve for the current sample
            current_predictions_list = predictions[current_ground_truth_query]
            if len(current_ground_truth_query_set)==0 :
              continue
            # Recall increments of this quantity each time a new correct prediction is encountered in the prediction list
            delta_recall = 1.0 / len(current_ground_truth_query_set)

            # Goes through the list of predictions
            encountered_positives = 0
            for idx, current_prediction in enumerate(current_predictions_list):
                # Each time a positive is encountered, compute the current precition and the area under the curve
                # since the last positive
                if current_prediction in current_ground_truth_query_set:
                    encountered_positives += 1
                    current_precision = encountered_positives / (idx + 1)
                    current_ap += current_precision * delta_recall

            m_ap += current_ap

        # Compute mean over all queries
        m_ap /= len(ground_truth)

        return m_ap

Creating query and test list from the training directory to evaluate our model

In [ ]:
train_query_list=[]
train_test_list=[]
query_ids_list=[]
d_test_truth={}
d_ids={}
i=1
for img,id_img,filename in training_data:
      if id_img not in query_ids_list:
        query_ids_list.append(id_img)
        train_query_list.append(filename)
        d_ids[id_img]=filename
        d_test_truth[d_ids[id_img]]=set()
      else:
        train_test_list.append(filename)
        d_test_truth[d_ids[id_img]].add(filename)
      i+=1
      if (i%256==0):
        print(i)

KeyboardInterrupt: ignored

Making the predictions

In [ ]:
d_pred_train,img_embd_query,img_embd_test=predict(train_query_list,train_dir,train_test_list,train_dir,net,device)

evaluating map

In [ ]:
evaluate_map(d_pred_train,d_test_truth)*100

In [ ]:
val_query_list=[]
val_test_list=[]
query_ids_list=[]
d_test_truth={}
d_ids={}
i=1
for img,id_img,filename in validation_data:
      if id_img not in query_ids_list:
        query_ids_list.append(id_img)
        val_query_list.append(filename)
        d_ids[id_img]=filename
        d_test_truth[d_ids[id_img]]=set()
      else:
        val_test_list.append(filename)
        d_test_truth[d_ids[id_img]].add(filename)
      i+=1
      if (i%256==0):
        print(i)

In [ ]:
d_pred_val,img_embd_query,img_embd_test=predict(val_query_list,train_dir,val_test_list,train_dir,net,device)

In [ ]:
evaluate_map(d_pred_val,d_test_truth)*100

# Making the predictions for test and query

creating test and query list

In [ ]:
query_test=[]
val_test=[]
for filename in os.listdir(test_dir):
    val_test.append(filename)
for filename in os.listdir(queries_dir):
    query_test.append(filename)

making the predictions

In [ ]:
d_pred_val,img_embd_query,img_embd_test=predict(query_test,queries_dir,val_test,test_dir,net,device)

0
256
512
768
1024
1280
1536
1792
2048
query_embedding_finished
0
256
512
768
1024
1280
1536
1792
2048
2304
2560
2816
3072
3328
3584
3840
4096
4352
4608
4864
5120
5376
5632
5888
6144
6400
6656
6912
7168
7424
7680
7936
8192
8448
8704
8960
9216
9472
9728
9984
10240
10496
10752
11008
11264
11520
11776
12032
12288
12544
12800
13056
13312
13568
13824
14080
14336
14592
14848
15104
15360
15616
15872
16128
16384
16640
16896
17152
17408
17664
17920
18176
18432
18688
18944
19200
19456
test_embedding_finished


writing in the prediction in a file


In [ ]:
f = open("/content/drive/MyDrive/projet deep learning/reid_test.txt", "w")
for key in d_pred_val.keys():
    s=', '.join(d_pred_val[key])
    f.write(str(key)+': '+s+'\n')
f.close()